# Create surveyplanner database

In [ ]:
from astropy.coordinates import Angle
import astropy.units as u
import numpy as np
from pandas import DataFrame

import constraints as c
from db import DBCreator, FieldManager
from fieldgrid import FieldGridIsoLat

## Pasiphae survey database

Create database:

In [2]:
db_name = 'test_strategy_init.sqlite3'
db = DBCreator(db_name)
created = db.create()

Database file exists. Overwrite (y) or cancel (enter)? y


Database 'test_strategy_init.sqlite3' created.
Table 'Fields' created.
Table 'Telescopes' created.
Table 'ParameterSets' created.
Table 'Constraints' created.
Table 'Parameters' created.
Table 'ParameterNames' created.
Table 'Observability' created.
Table 'ObservabilityStatus' created.
Table 'ObsWindows' created.
Table 'TimeRanges' created.
Table 'Observations' created.
Table 'Guidestars' created.
Table 'Filters' created.
View 'FieldsObs' created.
View 'Observable' created.
Constraints added to table 'Constraints'.
Statuses added to table 'ObservabilityStatus'.
Database creation finished.

Note: Next you need to add observatories, constraints, fields, guidestars, and observations.


Add observatories:

In [3]:
if created:
    name = 'Skinakas'
    lat = Angle('35:12:43 deg')
    lon = Angle('24:53:57 deg')
    height = 1750.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' added.


In [4]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' added.


Add constraints:

In [5]:
if created:
    # general limits:
    twilight = 'nautical'
    airmass_limit = c.AirmassLimit(2.)
    moon_distance = c.MoonDistance(10.)
    
    # Skinakas specific limits:
    hourangle_limit = c.HourangleLimit(5.33)
    
    # SAAO specific limits:
    polygon_ha = [-4, -4, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8]
    polygon_dec = [0.1, -52, -65, -85, -74.5, -70.0, -60, 0.1]
    ha_dec_limit = c.PolyHADecLimit(polygon_ha, polygon_dec)
    
    db.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))
    db.add_constraints('SAAO', twilight, constraints=(airmass_limit, moon_distance, ha_dec_limit))

Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'MoonDistance' for telescope 'Skinakas'.
Constraint 'AirmassLimit' for telescope 'SAAO'.
Constraint 'MoonDistance' for telescope 'SAAO'.
Constraint 'PolyHADecLimit' for telescope 'SAAO'.


Add Northern fields:

In [ ]:
fov = Angle(30 * u.arcmin)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(10 * u.arcsec)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(90 * u.deg)
dec_lim_south = Angle(0 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 83685


AttributeError: 'FieldGridIsoLat' object has no attribute 'save_params'

In [7]:
if created:
    db.add_fields(grid, 'Skinakas', active=True)

83685 fields added to database.                   


Add Southern fields:

In [8]:
fov = Angle(35 * u.arcmin)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(10 * u.arcsec)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(0 * u.deg)
dec_lim_south = Angle(-90 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 61630


In [9]:
if created:
    db.add_fields(grid, 'SAAO', active=True)

61630 fields added to database.                   


Add guide stars. For a simple demonstration we will add a fake guidestar to each field at the field center with a magnitude of 6:

In [10]:
if created:
    # get field IDs and center coordinates:
    manager = FieldManager(db_name)
    fields = DataFrame(manager.get_fields())
    
    # add fake guide stars:
    db.add_guidestars(
            fields['field_id'].to_numpy(), fields['center_ra'].to_numpy(), fields['center_dec'].to_numpy(), 
            np.ones(len(fields))*6, warn_missing=True)
    del fields

145315 new guidestars added to database.           
Checking for fields missing guidestars..
All fields have at least one guidestar associated.


Add observations. We add the same observation to each field:

In [11]:
db.add_observations(75., 2, 'r', check_for_duplicates=False)

Adding observations 0-999 of 145315 (0.0%)..

Filter 'r does not exist. Add it to data base? (y/n) y


145315 observation(s) added to data base.                 


In [12]:
db.dbstatus();

======= STORED IN DATABASE ========
Telescopes:                       2
Constraints telescope 1:          4
Constraints telescope 2:          4
-----------------------------------
Fields total:                145315
Fields telescope 1:           83685
Fields telescope 2:           61630
-----------------------------------
Observations:                145315
Pending observations:        145315
Fields w/o observations:          0
-----------------------------------
Guidestars:                  145315
Fields w/o guidestars:            0
-----------------------------------



## Test database with Pasiphae constraints

Create database:

In [13]:
db_name = 'test_planner_init.sqlite3'
db = DBCreator(db_name)
created = db.create()

Database file exists. Overwrite (y) or cancel (enter)? y


Database 'test_planner_init.sqlite3' created.
Table 'Fields' created.
Table 'Telescopes' created.
Table 'ParameterSets' created.
Table 'Constraints' created.
Table 'Parameters' created.
Table 'ParameterNames' created.
Table 'Observability' created.
Table 'ObservabilityStatus' created.
Table 'ObsWindows' created.
Table 'TimeRanges' created.
Table 'Observations' created.
Table 'Guidestars' created.
Table 'Filters' created.
View 'FieldsObs' created.
View 'Observable' created.
Constraints added to table 'Constraints'.
Statuses added to table 'ObservabilityStatus'.
Database creation finished.

Note: Next you need to add observatories, constraints, fields, guidestars, and observations.


Add observatories:

In [14]:
if created:
    name = 'Skinakas'
    lat = Angle('35:12:43 deg')
    lon = Angle('24:53:57 deg')
    height = 1750.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' added.


In [15]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' added.


Add constraints:

In [16]:
if created:
    # general limits:
    twilight = 'nautical'
    airmass_limit = c.AirmassLimit(2.)
    moon_distance = c.MoonDistance(10.)
    
    # Skinakas specific limits:
    hourangle_limit = c.HourangleLimit(5.33)
    
    # SAAO specific limits:
    polygon_ha = [-4, -4, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8]
    polygon_dec = [0.1, -52, -65, -85, -74.5, -70.0, -60, 0.1]
    ha_dec_limit = c.PolyHADecLimit(polygon_ha, polygon_dec)
    
    db.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))
    db.add_constraints('SAAO', twilight, constraints=(airmass_limit, moon_distance, ha_dec_limit))

Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'MoonDistance' for telescope 'Skinakas'.
Constraint 'AirmassLimit' for telescope 'SAAO'.
Constraint 'MoonDistance' for telescope 'SAAO'.
Constraint 'PolyHADecLimit' for telescope 'SAAO'.


Add Northern fields:

In [17]:
fov = Angle(5 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(90 * u.deg)
dec_lim_south = Angle(0 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 1097


In [18]:
if created:
    db.add_fields(grid, 'Skinakas', active=True)

1097 fields added to database.                   


Add Southern fields:

In [19]:
fov = Angle(7 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(0 * u.deg)
dec_lim_south = Angle(-90 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 547


In [20]:
if created:
    db.add_fields(grid, 'SAAO', active=True)

547 fields added to database.                   


Add guide stars. For a simple demonstration we will add a fake guidestar to each field at the field center with a magnitude of 6:

In [21]:
if created:
    # get field IDs and center coordinates:
    manager = FieldManager(db_name)
    fields = DataFrame(manager.get_fields())
    
    # add fake guide stars:
    db.add_guidestars(
            fields['field_id'].to_numpy(), fields['center_ra'].to_numpy(), fields['center_dec'].to_numpy(), 
            np.ones(len(fields))*6, warn_missing=True)
    del fields

1644 new guidestars added to database.           
Checking for fields missing guidestars..
All fields have at least one guidestar associated.


Add observations. We add the same observation to each field:

In [22]:
db.add_observations(75., 2, 'r', check_for_duplicates=False)

Adding observations 0-999 of 1644 (0.0%)..

Filter 'r does not exist. Add it to data base? (y/n) y


1644 observation(s) added to data base.                 


In [23]:
db.dbstatus();

======= STORED IN DATABASE ========
Telescopes:                       2
Constraints telescope 1:          4
Constraints telescope 2:          4
-----------------------------------
Fields total:                  1644
Fields telescope 1:            1097
Fields telescope 2:             547
-----------------------------------
Observations:                  1644
Pending observations:          1644
Fields w/o observations:          0
-----------------------------------
Guidestars:                    1644
Fields w/o guidestars:            0
-----------------------------------



## Test database with minimum constraints

Create database:

In [24]:
db_name = 'test_planner_min_constr_init.sqlite3'
db = DBCreator(db_name)
created = db.create()

Database file exists. Overwrite (y) or cancel (enter)? y


Database 'test_planner_min_constr_init.sqlite3' created.
Table 'Fields' created.
Table 'Telescopes' created.
Table 'ParameterSets' created.
Table 'Constraints' created.
Table 'Parameters' created.
Table 'ParameterNames' created.
Table 'Observability' created.
Table 'ObservabilityStatus' created.
Table 'ObsWindows' created.
Table 'TimeRanges' created.
Table 'Observations' created.
Table 'Guidestars' created.
Table 'Filters' created.
View 'FieldsObs' created.
View 'Observable' created.
Constraints added to table 'Constraints'.
Statuses added to table 'ObservabilityStatus'.
Database creation finished.

Note: Next you need to add observatories, constraints, fields, guidestars, and observations.


Add observatories:

In [25]:
if created:
    name = 'Skinakas'
    lat = Angle('35:12:43 deg')
    lon = Angle('24:53:57 deg')
    height = 1750.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' added.


In [26]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' added.


Add constraints:

In [27]:
if created:
    # general limits:
    twilight = 'nautical'
    airmass_limit = c.AirmassLimit(2.)
    hourangle_limit = c.HourangleLimit(5.33)
    
    db.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit))
    db.add_constraints('SAAO', twilight, constraints=(airmass_limit, hourangle_limit))

Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'AirmassLimit' for telescope 'SAAO'.
Constraint 'HourangleLimit' for telescope 'SAAO'.


Add Northern fields:

In [28]:
fov = Angle(5 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(90 * u.deg)
dec_lim_south = Angle(0 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 1097


In [29]:
if created:
    db.add_fields(grid, 'Skinakas', active=True)

1097 fields added to database.                   


Add Southern fields:

In [30]:
fov = Angle(7 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(0 * u.deg)
dec_lim_south = Angle(-90 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 547


In [31]:
if created:
    db.add_fields(grid, 'SAAO', active=True)

547 fields added to database.                   


Add guide stars. For a simple demonstration we will add a fake guidestar to each field at the field center with a magnitude of 6:

In [32]:
if created:
    # get field IDs and center coordinates:
    manager = FieldManager(db_name)
    fields = DataFrame(manager.get_fields())
    
    # add fake guide stars:
    db.add_guidestars(
            fields['field_id'].to_numpy(), fields['center_ra'].to_numpy(), fields['center_dec'].to_numpy(), 
            np.ones(len(fields))*6, warn_missing=True)
    del fields

1644 new guidestars added to database.           
Checking for fields missing guidestars..
All fields have at least one guidestar associated.


Add observations. We add the same observation to each field:

In [33]:
db.add_observations(75., 2, 'r', check_for_duplicates=False)

Adding observations 0-999 of 1644 (0.0%)..

Filter 'r does not exist. Add it to data base? (y/n) y


1644 observation(s) added to data base.                 


In [34]:
db.dbstatus();

======= STORED IN DATABASE ========
Telescopes:                       2
Constraints telescope 1:          3
Constraints telescope 2:          3
-----------------------------------
Fields total:                  1644
Fields telescope 1:            1097
Fields telescope 2:             547
-----------------------------------
Observations:                  1644
Pending observations:          1644
Fields w/o observations:          0
-----------------------------------
Guidestars:                    1644
Fields w/o guidestars:            0
-----------------------------------

